In [1]:
import os

from PIL import Image
import pandas as pd
from tqdm.auto import tqdm
import numpy as np
import pydicom
from pydicom.pixel_data_handlers.util import apply_voi_lut

In [2]:
import os

from PIL import Image
import pandas as pd
from tqdm.auto import tqdm
import numpy as np
import pydicom
from pydicom.pixel_data_handlers.util import apply_voi_lut
from numba import cuda
import gc
import torch
import sys
sys.path.append("./timmeffnetv2")

import time
import cv2
import PIL.Image
import random
from sklearn.metrics import accuracy_score
from tqdm.notebook import tqdm
import torch
from torch.utils.data import DataLoader, Dataset
import torch.nn as nn
from torch.nn import Parameter
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
import albumentations
import timm
import gc
from collections import OrderedDict
from warnings import filterwarnings
filterwarnings("ignore")
device = torch.device('cuda')


from glob import glob
import shutil, os
import matplotlib.pyplot as plt
from sklearn.model_selection import GroupKFold
import seaborn as sns

In [5]:
# 设置随机种子，以便实验复现

def set_seed(seed = 42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
set_seed()

'''
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False # set True to be faster
seed_everything(42)
'''

"\ndef seed_everything(seed):\n    random.seed(seed)\n    os.environ['PYTHONHASHSEED'] = str(seed)\n    np.random.seed(seed)\n    torch.manual_seed(seed)\n    torch.cuda.manual_seed(seed)\n    torch.backends.cudnn.deterministic = True\n    torch.backends.cudnn.benchmark = False # set True to be faster\nseed_everything(42)\n"

# 1. dcm to png

In [6]:
# from processing

In [7]:
# from https://www.kaggle.com/code/xhlulu/siim-covid-19-convert-to-jpg-256px?scriptVersionId=63196459
def read_xray(path, voi_lut = True, fix_monochrome = True):
    dicom = pydicom.read_file(path)

    if voi_lut:
        data = apply_voi_lut(dicom.pixel_array, dicom)
    else:
        data = dicom.pixel_array
               
    # depending on this value, X-ray may look inverted - fix that:
    if fix_monochrome and dicom.PhotometricInterpretation == "MONOCHROME1":
        data = np.amax(data) - data
        
    data = data - np.min(data)
    data = data / np.max(data)
    data = (data * 255).astype(np.uint8)
        
    return data

In [8]:
def resize(array, size, keep_ratio=False, resample=Image.LANCZOS):
    # Original from: https://www.kaggle.com/xhlulu/vinbigdata-process-and-resize-to-image
    im = Image.fromarray(array)
    
    if keep_ratio:
        im.thumbnail((size, size), resample)
    else:
        im = im.resize((size, size), resample)
    
    return im

In [9]:
image_id = []
dim0 = []
dim1 = []
splits = []

split = 'test'

save_dir = f'./tmp/{split}/'

os.makedirs(save_dir, exist_ok=True)
    
for dirname, _, filenames in tqdm(os.walk(f'./siim-covid19-detection/{split}')):
    for file in filenames:
        # set keep_ratio=True to have original aspect ratio
        xray = read_xray(os.path.join(dirname, file))
        im = resize(xray, size=512)  
        im.save(os.path.join(save_dir, file.replace('dcm', 'png')))

        image_id.append(file.replace('.dcm', ''))
        dim0.append(xray.shape[0])
        dim1.append(xray.shape[1])
        splits.append(split)

0it [00:00, ?it/s]

In [11]:
meta = pd.DataFrame.from_dict({'image_id': image_id, 'dim0': dim0, 'dim1': dim1, 'split': splits})
meta = meta[meta['split'] == 'test']
meta['image_id'] = meta['image_id'] + '_image'
meta.columns = ['id', 'dim0', 'dim1', 'split']
meta.to_csv('meta_test.csv', index=False)
meta

,id,dim0,dim1,split
0,3dcdfc352a06_image,3480,4248,test
1,c39146cbda47_image,3488,4256,test
2,951211f8e1bb_image,2320,2832,test
3,5e0e7acd9c7d_image,2800,3408,test
4,5b8ee5baa1d5_image,2320,2832,test
...,...,...,...,...
1258,064b37b01cd2_image,3480,4240,test
1259,02eceb0fc405_image,3488,4256,test
1260,16a0d37747d7_image,3520,4280,test
1261,aa55fc6c616c_image,3001,3001,test


# 2. 2-class Predict on Image Level

In [12]:
all_sub_df = pd.read_csv('./siim-covid19-detection/sample_submission.csv')
all_sub_df['id_last_str'] = all_sub_df["id"].str[-1]
all_sub_df

,id,PredictionString,id_last_str
0,00188a671292_study,negative 1 0 0 1 1,y
1,004bd59708be_study,negative 1 0 0 1 1,y
2,00508faccd39_study,negative 1 0 0 1 1,y
3,006486aa80b2_study,negative 1 0 0 1 1,y
4,00655178fdfc_study,negative 1 0 0 1 1,y
...,...,...,...
2472,46719b856de1_image,none 1 0 0 1 1,e
2473,31c07523a69a_image,none 1 0 0 1 1,e
2474,f77d7d1aebab_image,none 1 0 0 1 1,e
2475,ccc5b63ca96d_image,none 1 0 0 1 1,e


## Data Enhancement

In [13]:
# 为EfficientNet进行数据增强

## EfficientNet for 2-class Predict

In [14]:
# 用EfficientNet预测是否是none，不是none的再用Yolov5

## Yolov5 Predict

In [27]:
study_len = all_sub_df[all_sub_df['id_last_str'] == 'y'].shape[0]
study_len
yolov5_df = all_sub_df[study_len:].reset_index(drop=True)
yolov5_df = pd.merge(yolov5_df, meta, on='id', how='left')
yolov5_df = yolov5_df.drop(['PredictionString'], axis=1)
yolov5_df

,id,id_last_str,dim0,dim1,split
0,557a70442928_image,e,3320,3408,test
1,36141cda67ad_image,e,3480,4240,test
2,2413a23a5477_image,e,2320,2832,test
3,c263b1e9aa64_image,e,2801,2802,test
4,4fe0444d7fc5_image,e,2392,3014,test
...,...,...,...,...,...
1258,46719b856de1_image,e,2648,2867,test
1259,31c07523a69a_image,e,3488,4256,test
1260,f77d7d1aebab_image,e,3052,3012,test
1261,ccc5b63ca96d_image,e,2520,3032,test


In [29]:
dim = 512
test_dir = f'./tmp/{split}'
# shutil.copytree('./yolov5-master', './working/yolov5')
os.chdir('.')
import torch

In [ ]:
weights_dir = './best.pt'
test_dir = f'../tmp/{split}'
!python detect.py --weights $weights_dir\
--img 512\
--augment\
--conf 0.1\
--iou 0.5\
--source $test_dir\
--name test\
--save-txt --save-conf --exist-ok

In [ ]:
def yolo2voc(bboxes):
    """
    yolo => [xmid, ymid, w, h] (normalized)
    voc  => [x1, y1, x2, y1]

    """ 
    bboxes = bboxes.copy().astype(float) # otherwise all value will be 0 as voc_pascal dtype is np.int

    bboxes[..., [0, 1]] = bboxes[..., [0, 1]] - bboxes[..., [2, 3]]/2
    bboxes[..., [2, 3]] = bboxes[..., [0, 1]] + bboxes[..., [2, 3]]
    
    return bboxes

def yolo2voc1(image_height, image_width, bboxes):
    bboxes = bboxes.copy().astype(float) # otherwise all value will be 0 as voc_pascal dtype is np.int

    bboxes[..., [0, 2]] = bboxes[..., [0, 2]]* image_width
    bboxes[..., [1, 3]] = bboxes[..., [1, 3]]* image_height
    
    return bboxes

In [ ]:
from ensemble_boxes import weighted_boxes_fusion
image_ids = []
PredictionStrings = []

for i in range(len(yolov5_df)):
    boxes_list = []
    scores_list = []
    labels_list = []
    is_exist1 = False
    is_exist2 = True
    
    iou_thr = 0.5
    skip_box_thr = 0.1
    image_id = yolov5_df.loc[i,'id'].replace('_image', '')
    w, h = yolov5_df.loc[yolov5_df.id==image_id,['dim1', 'dim0']].values[0]
        
    file_path = './yolov5-master/runs/detect/test/labels/{}.txt'.format(image_id)
    if os.path.exists(file_path):
        f = open(file_path, 'r')
        data = np.array(f.read().replace('\n', ' ').strip().split(' ')).astype(np.float32).reshape(-1, 6)
        data = data[:, [0, 5, 1, 2, 3, 4]]
        data[:,2:] = yolo2voc(data[:, 2:])
        data2 = data
    else:
        is_exist2 = False
        
    if is_exist2==False:
        continue

    bboxes = list(np.round(np.concatenate((data2[:, :2], np.round(yolo2voc1(h, w, data2[:, 2:]))), axis =1).reshape(-1), 12).astype(str))
    for idx in range(len(bboxes)):
        bboxes[idx] = str(int(float(bboxes[idx]))) if idx%6!=1 else bboxes[idx]
    image_ids.append(image_id)
    PredictionStrings.append(' '.join(bboxes))

image_sub_2_df = pd.DataFrame({'id':image_ids + '_image',
                        'PredictionString':PredictionStrings})

In [ ]:
image_sub_2_df

In [ ]:
yolov5_df = pd.merge(yolov5_df, image_sub_2_df, on = 'id', how = 'left').fillna("none 1 0 0 1 1")
yolov5_df = yolov5_df[['id', 'PredictionString']]
yolov5_df

In [ ]:
for i in range(yolov5_df.shape[0]):
    if yolov5_df.loc[i,'PredictionString'] == "none 1 0 0 1 1":
        continue
    sub_df_split = yolov5_df.loc[i,'PredictionString'].split()
    sub_df_list = []
    for j in range(int(len(sub_df_split) / 6)):
        sub_df_list.append('opacity')
        sub_df_list.append(sub_df_split[6 * j + 1])
        sub_df_list.append(sub_df_split[6 * j + 2])
        sub_df_list.append(sub_df_split[6 * j + 3])
        sub_df_list.append(sub_df_split[6 * j + 4])
        sub_df_list.append(sub_df_split[6 * j + 5])
    yolov5_df.loc[i,'PredictionString'] = ' '.join(sub_df_list)

yolov5_df['none'] = df_2class['none'] 
for i in range(yolov5_df.shape[0]):
    if yolov5_df.loc[i,'PredictionString'] != 'none 1 0 0 1 1':
        yolov5_df.loc[i,'PredictionString'] = yolov5_df.loc[i,'PredictionString'] + ' none ' + str(yolov5_df.loc[i,'none']) + ' 0 0 1 1'

yolov5_df = yolov5_df[['id', 'PredictionString']] 
yolov5_df

In [ ]:
image_all_df = yolov5_df

## MMDET Predict

# 3. 4-class Predict on Study Level

# 4. Merge Study and Image